In [7]:
import tkinter as tk
from tkinter import ttk
import psycopg2
import pandas as pd
import sqlite3 

db_creds = pd.read_csv('/users/zack burnside/desktop/database_creds.csv')

tournament_data = pd.read_csv('/users/zack burnside/desktop/tournament_info.csv')
player_choice = pd.read_csv('/users/zack burnside/desktop/player_db.csv')
usernames = names = [
    "Z",
    "Timmy Chips",
    "Goob",
    "C",
    "Steve",
    "Sus",
    "DV3",
    "Jlo",
    "Ty Dolla $$$",
    "Ryan",
    "Mike D",
    "Mando",
    "Eddie Buckets",
    "Garrett",
    "T Sully"
]

tournament_names = list(tournament_data['tournament_name'])
player_picks = list(player_choice['player_name'])
original_tournament_names = tournament_names.copy()

def submit():
    user_name = user_name_var.get()
    tournament_name = tournament_name_var.get()
    player_pick = player_pick_var.get()
    
    # Connect to SQLite database
    conn = sqlite3.connect('data.db')
    c = conn.cursor()
    
    # Create table if not exists
    c.execute('''CREATE TABLE IF NOT EXISTS picks
                 (id INTEGER PRIMARY KEY, user_name TEXT, tournament_name TEXT, player_pick TEXT)''')
    
    # Insert data into the table
    c.execute('''INSERT INTO picks (user_name, tournament_name, player_pick)
                 VALUES (?, ?, ?)''', (user_name, tournament_name, player_pick))
    
    print({'Username' : user_name,
           'Tournament Name' : tournament_name,
           'Player Pick' : player_pick
           })
    
    conn.commit()
    conn.close()

def show_standings():
    # Connect to PostgreSQL database
    conn = psycopg2.connect(
        dbname = db_creds['dbname'][0],
        user = db_creds['user'][0],
        password = db_creds['password'][0],
        host = db_creds['host'][0],
        port = "5433"
    )
    cursor = conn.cursor()

    # Execute a SELECT query to retrieve all rows from the 'picks' table
    cursor.execute("SELECT * FROM new_schema.standing_results")

    # Fetch all rows from the result set
    rows = cursor.fetchall()

    # Create a new window for standings
    standings_window = tk.Toplevel(root)
    standings_window.title("Standings")

    # Create a Treeview widget
    tree = ttk.Treeview(standings_window, columns=("User Name", "Total Money"), show="headings")
    tree.heading("User Name", text="User Name")
    tree.heading("Total Money", text="Total Money")
    tree.pack()

    # Insert data into the Treeview
    for row in rows:
        tree.insert("", "end", values=row)

    # Close the connection
    conn.close()

original_usernames = usernames.copy()
original_player_picks = player_picks.copy()

def display_standings():
    # Retrieve standings data
    standings_data = show_standings()

    if standings_data:  # Check if data is retrieved successfully
        # Create a new window for standings
        standings_window = tk.Toplevel(root)
        standings_window.title("Standings")

        # Create a Treeview widget
        tree = ttk.Treeview(standings_window, columns=("User Name", "Total Money"), show="headings")
        tree.heading("User Name", text="User Name")
        tree.heading("Total Money", text="Total Money")
        tree.pack()

        # Insert data into the Treeview
        for row in standings_data:
            tree.insert("", "end", values=row)
    else:
        # Display an error message if no data is retrieved
        error_message = "Error: Unable to retrieve standings data."
        tk.messagebox.showerror("Error", error_message)

def filter_dropdown(event):
    # Get the current text in the combobox
    current_text = event.widget.get().lower()
    # Determine which dropdown is being filtered
    if event.widget == tournament_name_dropdown:
        full_values = original_tournament_names
    elif event.widget == player_pick_dropdown:
        full_values = original_player_picks
    else:
        full_values = original_usernames  # Assuming this is the user names dropdown
    
    # Filter values based on current text
    filtered_values = [value for value in full_values if current_text in value.lower()]
    
    # If there are no filtered values, restore original values
    if not filtered_values:
        event.widget['values'] = full_values
    else:
        # Update combobox values with filtered values
        event.widget['values'] = filtered_values

# Create main window
root = tk.Tk()
root.title("PGA Pool Picker")

# Create labels
user_name_label = ttk.Label(root, text="User Name:")
user_name_label.grid(row=0, column=0, padx=5, pady=5)

tournament_name_label = ttk.Label(root, text="Tournament Name:")
tournament_name_label.grid(row=1, column=0, padx=5, pady=5)

player_pick_label = ttk.Label(root, text="Player Pick:")
player_pick_label.grid(row=2, column=0, padx=5, pady=5)

# Create drop down menus
user_name_var = tk.StringVar()
user_name_dropdown = ttk.Combobox(root, textvariable=user_name_var)
user_name_dropdown['values'] = usernames
user_name_dropdown.grid(row=0, column=1, padx=5, pady=5)
user_name_dropdown.bind("<KeyRelease>", filter_dropdown)

tournament_name_var = tk.StringVar()
tournament_name_dropdown = ttk.Combobox(root, textvariable=tournament_name_var)
tournament_name_dropdown['values'] = list(tournament_data['tournament_name'])
tournament_name_dropdown.grid(row=1, column=1, padx=5, pady=5)
tournament_name_dropdown.bind("<KeyRelease>", filter_dropdown)


player_pick_var = tk.StringVar()
player_pick_dropdown = ttk.Combobox(root, textvariable=player_pick_var)
player_pick_dropdown['values'] = list(player_choice['player_name'])
player_pick_dropdown.grid(row=2, column=1, padx=5, pady=5)
player_pick_dropdown.bind("<KeyRelease>", filter_dropdown)

# Create submit button
submit_button = ttk.Button(root, text="Submit", command=submit)
submit_button.grid(row=3, column=0, columnspan=2, padx=5, pady=5)

# Standings Button
show_standings_button = tk.Button(root, text="Show Standings", command=display_standings)
show_standings_button.grid(row=0, column=5, pady=10)
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Zack Burnside\anaconda3\New folder\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Zack Burnside\AppData\Local\Temp\ipykernel_1908\2936614174.py", line 116, in display_standings
    tk.messagebox.showerror("Error", error_message)
AttributeError: module 'tkinter' has no attribute 'messagebox'


KeyboardInterrupt: 